# 🧠 Neuro-TabPFN v0.3 - Pipeline

**Pipeline completo para predicción causal de outcomes en stroke:**
1. Falta actualizar
2. 
3. VAE Adaptativo (cualquier shape)
4. TabICL Two-Stage
5. Do-Loss Training

In [1]:
# SETUP
import os, sys

ROOT = "H:/My Drive/Debbuging Neuro"
DATA_DIR = f"{ROOT}/Data"
REPO_DIR = f"{ROOT}/individualized_prescriptive_inference"
RESULTS_DIR = f"{ROOT}/results_intersynth_full"
REP_DIR = os.path.join(RESULTS_DIR, "representations")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

os.chdir(ROOT)
print(f"Working dir: {os.getcwd()}")

Working dir: H:\My Drive\Debbuging Neuro


In [ ]:
# Elige condición: lesion vs disco, genetics vs receptor, k-fold, threshold
lesion_deficit_thresh = "0.05"
gene_or_receptor = "receptor"   # o "genetics"
kfold = 0
input_type = "lesion"           # o "disco" si tienes disconnectomes

stem = f"{input_type}_{lesion_deficit_thresh}_{gene_or_receptor}_{kfold}"
train_pkl = glob.glob(os.path.join(REP_DIR, stem, f"train_{input_type}_{lesion_deficit_thresh}_{kfold}.pkl"))
test_pkl  = glob.glob(os.path.join(REP_DIR, stem, f"test_{input_type}_{lesion_deficit_thresh}_{kfold}.pkl"))
centroids_json = os.path.join(REP_DIR, stem, "centroids.json")


In [3]:
#%%capture
#!pip install torch torchvision monai nibabel pandas numpy matplotlib scikit-learn datasets "abagen[io]"

In [ ]:
# Clonar repo y deps
%%capture

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/high-dimensional/individualized_prescriptive_inference.git $REPO_DIR
        
%cd $REPO_DIR
!pip install -q -r requirements.txt

In [4]:
import os, random, warnings, json
from pathlib import Path
from typing import Optional, Dict, Tuple, List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
from scipy.ndimage import zoom
from datasets import load_dataset
from monai.transforms import Spacing
from monai.data import MetaTensor
import shutil, glob, json, textwrap, subprocess

plt.style.use('seaborn-v0_8-whitegrid')
warnings.filterwarnings('ignore')

In [6]:
# CONFIG
class Config:
    SEED = 42
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
    HF_DATASET = "hugging-science/isles24-stroke"
    N_SUBSET = 500
    TARGET_SHAPE = (96, 112, 96)
    TARGET_SPACING = (2.0, 2.0, 2.0)
    LATENT_DIM = 50
    VAE_BASE_CHANNELS = 32
    VAE_EPOCHS = 30
    VAE_BATCH_SIZE = 16
    VAE_LR = 1e-3
    VAE_BETA = 1.0
    SCM_EFFECT = 5.0
    SCM_WEIGHT_Z = 0.5
    SCM_WEIGHT_T = 2.0
    D_MODEL = 128
    N_HEAD = 4
    N_LAYERS_COL = 2
    N_LAYERS_ROW = 4
    DIM_FEEDFORWARD = 512
    DROPOUT = 0.1
    LR_TABICL = 5e-4
    SYN_BATCH = 128
    SYN_SEQ = 64
    DO_STEPS = 500

cfg = Config()
random.seed(cfg.SEED); np.random.seed(cfg.SEED); torch.manual_seed(cfg.SEED)
print(f" Device: {cfg.DEVICE}, Shape: {cfg.TARGET_SHAPE}")

 Device: cpu, Shape: (96, 112, 96)


In [7]:
# Descargar lesiones (4119 máscaras MNI)
LESIONS_ZIP = f"{DATA_DIR}/lesions.zip"
if not os.path.exists(LESIONS_ZIP):
    !wget -q https://github.com/high-dimensional/individualized_prescriptive_inference/raw/main/lesions.zip -O $LESIONS_ZIP
LESIONS_DIR = f"{DATA_DIR}/lesions_full"
if not os.path.exists(LESIONS_DIR):
    !unzip -q $LESIONS_ZIP -d $LESIONS_DIR
print("Total máscaras:", len(glob.glob(os.path.join(LESIONS_DIR, "*.nii*"))))

# Añade ruta a disconnectomes si los tienes:
DISCO_DIR = ""  # deja vacío si no tienes disconnectomes

 Loading ISLES'24...


Resolving data files:   0%|          | 0/149 [00:00<?, ?it/s]

   30/149...
   60/149...
   90/149...
   120/149...
 Loaded 149 cases. Shape: (149, 1, 96, 112, 96)


In [ ]:
# 3) Ejecutar representación (embeddings NM/PCA, etc.)
#    Se usa por defaults del script (k-folds, latents, nmf/pca ON).
rep_cmd = textwrap.dedent(f"""
python software/representation.py \
  --lesionpath {LESIONS_DIR} \
  --discopath "{DISCO_DIR}" \
  --savepath {RESULTS_DIR}/representations
""").strip()
print("Running representation.py ...")
subprocess.run(rep_cmd, shell=True, check=True)

In [ ]:
# 4) Ground-truth deficit modelling (crea train/test pkl por déficit)
#    Usa los atlases del repo (functional_parcellation/2mm_parcellations).

def_cmd = textwrap.dedent(f"""
python software/deficit_modelling.py \
  --path {RESULTS_DIR}/representations \
  --lesionpath {LESIONS_DIR} \
  --discopath "{DISCO_DIR}" \
  --latent_list 2 4 8 16 32 64 128 256 \
  --kfold_deficits 10 \
  --roi_threshs 0.05 \
  --names genetics receptor \
  --run_ae False \
  --run_vae False \
  --run_nmf True \
  --run_pca True
""").strip()
print("Running deficit_modelling.py ...")
subprocess.run(def_cmd, shell=True, check=True)

In [ ]:
# 5) Prescriptive simulations (pipeline completo)
#    Usa los defaults amplios: genética + receptores, lesión + disco (si disco existe),
#    múltiples bias/TE/RE, 16 déficits, varios bottlenecks.

presc_cmd = textwrap.dedent(f"""
python software/prescription.py \
  --savepath {RESULTS_DIR} \
  --loadpath {RESULTS_DIR}/representations \
  --k 0 1 2 3 4 5 6 7 8 9 \
  --gene_or_receptor genetics receptor \
  --lesion_or_disconnectome lesion {"disco" if DISCO_DIR else ""} \
  --lesion_deficit_thresh 0.05 0.05 \
  --deficits 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 \
  --biasdegree 0 0.25 0.5 0.75 1 \
  --biastype observed unobserved \
  --te 1 0.75 0.5 0.25 \
  --re 0 0.25 0.5 0.75 \
  --bottlenecks 0 2 4 8 16 32 64 128 \
  --simpleatlases False all_territories major_arterial_territories major_arterial_territories_lat major_territories clusters_lat \
  --simpleatlas_argmaxs True \
  --vols True False \
  --centroids False \
  --ml_models logistic_regression extra_trees xgb \
  --use_vae False \
  --use_ae False \
  --use_nmf True \
  --use_pca True
""").strip()
print("Running prescription.py ... (esto puede tardar MUCHO)")
subprocess.run(presc_cmd, shell=True, check=True)

In [ ]:
# 6) Listar salidas
import pandas as pd, glob, os
out_files = glob.glob(os.path.join(RESULTS_DIR, "prescription", "*.pkl"))
print("Archivos generados:", len(out_files))
if out_files:
    df_sample = pd.read_pickle(out_files[0])
    print(df_sample.head())

In [ ]:
assert train_pkl and test_pkl, "No se encontraron pickles; revisa rutas/flags de representación."
train_df = pd.read_pickle(train_pkl[0])
test_df  = pd.read_pickle(test_pkl[0])

print("Cols disponibles:", train_df.columns[:20], "... total:", len(train_df.columns))
print("Shape train/test:", train_df.shape, test_df.shape)

# Ejemplo: elegir un embedding ya calculado (p.ej. PCA 32D si se activó)
# Cambia 'pca' por 'nmf'/'vae_means'/'ae' según lo que activaste
bottleneck = 32
feat_col = f"pca_{input_type}_{bottleneck}_K{kfold}"
if feat_col in train_df.columns:
    import numpy as np
    X_train = np.stack(train_df[feat_col]).astype("float32")
    X_test  = np.stack(test_df[feat_col]).astype("float32")
else:
    raise ValueError(f"No se encontró la columna {feat_col}; verifica los parámetros de representación.")

# Etiquetas de outcome sintetizado (y_true en test) y grupos W en train (asignación de tratamiento simulada)
y_train = train_df["y_true"].to_numpy() if "y_true" in train_df else None
y_test  = test_df["y_true"].to_numpy()  if "y_true" in test_df  else None
W_train = train_df["group"].to_numpy() if "group" in train_df else None

print("X_train:", X_train.shape, "y_train:", None if y_train is None else y_train.shape)

In [8]:
# INTERSYNTH
class MultiReceptorInterSynth:
    RECEPTORS = ['D1', 'D2', '5HT1A', '5HT2A', 'GABA', 'mGluR5']
    
    def __init__(self, shape):
        self.shape = shape
        self.weight_maps = {r: self._gen_map(r) for r in self.RECEPTORS}
        print(f" {len(self.weight_maps)} receptor maps")
    
    def _gen_map(self, receptor):
        H, W, D = self.shape
        z, y, x = np.linspace(-1,1,H), np.linspace(-1,1,W), np.linspace(-1,1,D)
        Z, Y, X = np.meshgrid(z, y, x, indexing='ij')
        R = np.sqrt(X**2 + Y**2 + Z**2)
        base = np.exp(-3 * (R - 0.7)**2) * (0.5 + 0.5*Y) * (0.5 + 0.5*Z)
        if receptor in ['D1', 'D2']:
            mod = np.exp(-5*(X**2+Y**2)) * np.exp(-2*(Z+0.3)**2)
        elif receptor in ['5HT1A', '5HT2A']:
            mod = 0.5 + 0.5*Y
        else:
            mod = base
        result = base * mod
        return ((result - result.min()) / (result.max() - result.min() + 1e-8)).astype(np.float32)
    
    def compute_composite(self, masks, normalize=True):
        if masks.ndim == 5: masks = masks[:, 0]
        masks = (masks > 0.5).astype(np.float32)
        scores = []
        for w in self.weight_maps.values():
            s = (torch.from_numpy(masks).float() * torch.from_numpy(w).float()).sum(dim=(1,2,3)).numpy()
            scores.append(s)
        composite = np.mean(scores, axis=0)
        if normalize:
            composite = (composite - composite.min()) / (composite.max() - composite.min() + 1e-8)
        return composite

intersynth = MultiReceptorInterSynth(cfg.TARGET_SHAPE)
Y_intersynth = intersynth.compute_composite(masks)
print(f"Y_intersynth: [{Y_intersynth.min():.3f}, {Y_intersynth.max():.3f}]")

 6 receptor maps
Y_intersynth: [0.000, 1.000]


In [9]:
# DATASET 
class MaskDataset(Dataset):
    def __init__(self, masks_np):
        assert masks_np.ndim == 5 and masks_np.shape[1] == 1, "Expected [N,1,H,W,D]"
        self.m = torch.from_numpy(masks_np).float()
    def __len__(self): return len(self.m)
    def __getitem__(self, idx): return self.m[idx]

dataset = MaskDataset(masks)
dataloader = DataLoader(dataset, batch_size=cfg.VAE_BATCH_SIZE, shuffle=True)
print(f" Batch shape: {next(iter(dataloader)).shape}")

 Batch shape: torch.Size([16, 1, 96, 112, 96])


In [10]:
# VAE ADAPTATIVO
class ResBlock3D(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv3d(in_ch, out_ch, 3, padding=1)
        self.conv2 = nn.Conv3d(out_ch, out_ch, 3, padding=1)
        self.bn1, self.bn2 = nn.BatchNorm3d(out_ch), nn.BatchNorm3d(out_ch)
        self.skip = nn.Conv3d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()
    def forward(self, x):
        return F.leaky_relu(self.bn2(self.conv2(F.leaky_relu(self.bn1(self.conv1(x)), 0.2))) + self.skip(x), 0.2)

class AdaptiveEncoder3D(nn.Module):
    def __init__(self, input_shape, latent_dim=64, base_ch=32):
        super().__init__()
        self.input_shape = input_shape
        self.blocks = nn.ModuleList([
            nn.Sequential(nn.Conv3d(1, base_ch, 4, 2, 1), nn.BatchNorm3d(base_ch), nn.LeakyReLU(0.2)),
            ResBlock3D(base_ch, base_ch),
            nn.Sequential(nn.Conv3d(base_ch, base_ch*2, 4, 2, 1), nn.BatchNorm3d(base_ch*2), nn.LeakyReLU(0.2)),
            ResBlock3D(base_ch*2, base_ch*2),
            nn.Sequential(nn.Conv3d(base_ch*2, base_ch*4, 4, 2, 1), nn.BatchNorm3d(base_ch*4), nn.LeakyReLU(0.2)),
            ResBlock3D(base_ch*4, base_ch*4),
            nn.Sequential(nn.Conv3d(base_ch*4, base_ch*8, 4, 2, 1), nn.BatchNorm3d(base_ch*8), nn.LeakyReLU(0.2)),
            ResBlock3D(base_ch*8, base_ch*8),
        ])
        self.feature_size, self.feature_shape = self._get_size()
        self.fc_mu = nn.Linear(self.feature_size, latent_dim)
        self.fc_logvar = nn.Linear(self.feature_size, latent_dim)
        print(f"   Encoder: {self.feature_shape}={self.feature_size}")
    
    def _get_size(self):
        with torch.no_grad():
            x = torch.zeros(1, 1, *self.input_shape)
            for b in self.blocks: x = b(x)
            return x.numel(), tuple(x.shape[1:])
    
    def forward(self, x):
        for b in self.blocks: x = b(x)
        x = x.view(x.size(0), -1)
        return self.fc_mu(x), self.fc_logvar(x)

class AdaptiveDecoder3D(nn.Module):
    def __init__(self, latent_dim, feature_shape, output_shape, base_ch=32):
        super().__init__()
        self.feature_shape, self.output_shape = feature_shape, output_shape
        self.fc = nn.Linear(latent_dim, int(np.prod(feature_shape)))
        ch = base_ch * 8
        self.blocks = nn.ModuleList([
            nn.Sequential(nn.ConvTranspose3d(ch, ch//2, 4, 2, 1), nn.BatchNorm3d(ch//2), nn.ReLU()),
            ResBlock3D(ch//2, ch//2),
            nn.Sequential(nn.ConvTranspose3d(ch//2, ch//4, 4, 2, 1), nn.BatchNorm3d(ch//4), nn.ReLU()),
            ResBlock3D(ch//4, ch//4),
            nn.Sequential(nn.ConvTranspose3d(ch//4, ch//8, 4, 2, 1), nn.BatchNorm3d(ch//8), nn.ReLU()),
            ResBlock3D(ch//8, ch//8),
            nn.Sequential(nn.ConvTranspose3d(ch//8, 1, 4, 2, 1), nn.Sigmoid())
        ])
        self._out = self._get_out()
        self.needs_resize = self._out != output_shape
    
    def _get_out(self):
        with torch.no_grad():
            x = torch.zeros(1, *self.feature_shape)
            for b in self.blocks: x = b(x)
            return tuple(x.shape[2:])
    
    def forward(self, z):
        x = self.fc(z).view(-1, *self.feature_shape)
        for b in self.blocks: x = b(x)
        if self.needs_resize: x = F.interpolate(x, size=self.output_shape, mode='trilinear', align_corners=False)
        return x

class AdaptiveVAE3D(nn.Module):
    def __init__(self, input_shape, latent_dim=64, base_ch=32):
        super().__init__()
        print(f" VAE for {input_shape}")
        self.encoder = AdaptiveEncoder3D(input_shape, latent_dim, base_ch)
        self.decoder = AdaptiveDecoder3D(latent_dim, self.encoder.feature_shape, input_shape, base_ch)
        print(f" VAE: {sum(p.numel() for p in self.parameters()):,} params")
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = mu + torch.randn_like(mu) * torch.exp(0.5 * logvar)
        return self.decoder(z), mu, logvar
    
    def encode(self, x): return self.encoder(x)[0]

def vae_loss(recon, original, mu, logvar, beta=1.0):
    # CRITICAL: verify shapes match
    assert recon.shape == original.shape, f"Shape mismatch: {recon.shape} vs {original.shape}"
    bs = original.size(0)
    recon_l = F.binary_cross_entropy(recon, original, reduction='sum') / bs
    kl_l = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / bs
    return recon_l + beta * kl_l, recon_l, kl_l

vae = AdaptiveVAE3D(cfg.TARGET_SHAPE, cfg.LATENT_DIM, cfg.VAE_BASE_CHANNELS).to(cfg.DEVICE)
with torch.no_grad():
    test = next(iter(dataloader))[:2].to(cfg.DEVICE)
    recon, mu, logvar = vae(test)
    loss, _, _ = vae_loss(recon, test, mu, logvar)
    print(f"\n Test: {test.shape}->{recon.shape}, loss={loss.item():.4f}")
print(" VAE OK!")

 VAE for (96, 112, 96)
   Encoder: (256, 6, 7, 6)=64512
 VAE: 21,118,245 params

 Test: torch.Size([2, 1, 96, 112, 96])->torch.Size([2, 1, 96, 112, 96]), loss=945822.7500
 VAE OK!


In [ ]:
# TRAIN VAE
def train_vae(vae, dl, epochs=cfg.VAE_EPOCHS):
    opt = optim.Adam(vae.parameters(), lr=cfg.VAE_LR)
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', factor=0.5, patience=5)
    history = {'loss': [], 'dice': []}
    vae.train()
    print(f"\n Training VAE {epochs} epochs...")
    for ep in range(epochs):
        total_l, total_d, n = 0, 0, 0
        for batch in dl:
            batch = batch.to(cfg.DEVICE)
            opt.zero_grad()
            recon, mu, logvar = vae(batch)
            loss, _, _ = vae_loss(recon, batch, mu, logvar, cfg.VAE_BETA)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(vae.parameters(), 1.0)
            opt.step()
            total_l += loss.item()
            p, t = (recon > 0.5).float(), (batch > 0.5).float()
            total_d += (2.0 * (p * t).sum() / (p.sum() + t.sum() + 1e-8)).item()
            n += 1
        history['loss'].append(total_l/n)
        history['dice'].append(total_d/n)
        sched.step(total_l/n)
        if (ep+1) % 10 == 0: print(f"   Ep {ep+1}: Loss={total_l/n:.4f}, Dice={total_d/n:.4f}")
    print(f" Final Dice: {history['dice'][-1]:.4f}")
    return history

vae_history = train_vae(vae, dataloader)


 Training VAE 30 epochs...
   Ep 10: Loss=22485.3082, Dice=0.1311


In [ ]:
# EXTRACT LATENTS
@torch.no_grad()
def extract_latents(vae, dl):
    vae.eval()
    return np.concatenate([vae.encode(b.to(cfg.DEVICE)).cpu().numpy() for b in dl])

Z = extract_latents(vae, dataloader)
print(f" Z: {Z.shape}, mean={Z.mean():.4f}, std={Z.std():.4f}")

In [ ]:
# TABICL TWO-STAGE
class LearnablePE(nn.Module):
    def __init__(self, d, max_len=512, drop=0.1):
        super().__init__()
        self.pe = nn.Parameter(torch.randn(1, max_len, d) * 0.02)
        self.drop = nn.Dropout(drop)
    def forward(self, x): return self.drop(x + self.pe[:, :x.size(1), :])

class FeatureEmbed(nn.Module):
    def __init__(self, d, n_feat, drop=0.1):
        super().__init__()
        self.n_feat = n_feat
        self.proj = nn.Linear(1, d)
        self.feat_emb = nn.Parameter(torch.randn(1, n_feat, 1, d) * 0.02)
        self.drop = nn.Dropout(drop)
    def forward(self, x):
        B, S, F = x.shape
        x = x.permute(0, 2, 1).unsqueeze(-1)
        x = self.proj(x) + self.feat_emb
        return self.drop(x.view(B * F, S, -1))

class ColBlock(nn.Module):
    def __init__(self, d, nh, ff, drop):
        super().__init__()
        layer = nn.TransformerEncoderLayer(d, nh, ff, drop, batch_first=True, activation='gelu', norm_first=True)
        self.enc = nn.TransformerEncoder(layer, 1)
    def forward(self, x): return self.enc(x)

class RowBlock(nn.Module):
    def __init__(self, d, n_feat, nh, nl, ff, drop):
        super().__init__()
        self.n_feat, self.d = n_feat, d
        rd = d * n_feat
        self.pe = LearnablePE(rd, drop=drop)
        layer = nn.TransformerEncoderLayer(rd, nh, ff, drop, batch_first=True, activation='gelu', norm_first=True)
        self.enc = nn.TransformerEncoder(layer, nl)
    def forward(self, h):
        B, S, F, D = h.shape
        return self.enc(self.pe(h.view(B, S, F * D)))

class TabICLTwoStage(nn.Module):
    def __init__(self, n_feat, d=128, nh_c=4, nh_r=4, nl_c=2, nl_r=4, ff=512, drop=0.1):
        super().__init__()
        self.n_feat, self.d = n_feat, d
        self.embed = FeatureEmbed(d, n_feat, drop)
        self.col_blks = nn.ModuleList([ColBlock(d, nh_c, ff, drop) for _ in range(nl_c)])
        self.row_blk = RowBlock(d, n_feat, nh_r, nl_r, ff*2, drop)
        self.head = nn.Sequential(
            nn.LayerNorm(d * n_feat),
            nn.Linear(d * n_feat, d), nn.GELU(), nn.Dropout(drop),
            nn.Linear(d, 1)
        )
        for p in self.parameters():
            if p.dim() > 1: nn.init.xavier_uniform_(p)
        print(f" TabICL: {sum(p.numel() for p in self.parameters()):,} params")
    
    def forward(self, x):
        B, S, F = x.shape
        h = self.embed(x)
        for blk in self.col_blks: h = blk(h)
        h = h.view(B, F, S, self.d).permute(0, 2, 1, 3).contiguous()
        return self.head(self.row_blk(h))

n_feat = cfg.LATENT_DIM + 2
model = TabICLTwoStage(n_feat, cfg.D_MODEL, cfg.N_HEAD, cfg.N_HEAD, cfg.N_LAYERS_COL, cfg.N_LAYERS_ROW, cfg.DIM_FEEDFORWARD, cfg.DROPOUT).to(cfg.DEVICE)
with torch.no_grad():
    tx = torch.randn(2, 10, n_feat).to(cfg.DEVICE)
    print(f" {tx.shape}->{model(tx).shape}")
print(" TabICL OK!")

In [ ]:
# DO-LOSS TRAINING
def make_batch(bs=cfg.SYN_BATCH, sl=cfg.SYN_SEQ, nf=cfg.LATENT_DIM, real_Z=None):
    if real_Z is None:
        z = torch.randn(bs, sl, nf, device=cfg.DEVICE)
    else:
        idx = torch.randint(0, real_Z.shape[0], (bs*sl,), device=cfg.DEVICE)
        z = real_Z[idx].view(bs, sl, -1)
        if z.shape[-1] < nf: z = torch.cat([z, torch.zeros(bs, sl, nf - z.shape[-1], device=cfg.DEVICE)], -1)
        elif z.shape[-1] > nf: z = z[:, :, :nf]
    logits = cfg.SCM_WEIGHT_T * z[:,:,0] + torch.randn_like(z[:,:,0])
    t = torch.bernoulli(torch.sigmoid(logits)).unsqueeze(-1)
    y = cfg.SCM_EFFECT * t - cfg.SCM_WEIGHT_Z * z[:,:,0:1] + 0.1 * torch.randn_like(t)
    return z, t, y

def do_step(model, opt, loss_fn, z, t, y):
    model.train()
    Zc, Zq = z[:,:-1,:], z[:,-1:,:]
    Tc, Tq = t[:,:-1,:], torch.bernoulli(torch.full_like(t[:,-1:,:], 0.5))
    Yc = y[:,:-1,:]
    Yq_true = cfg.SCM_EFFECT * Tq - cfg.SCM_WEIGHT_Z * Zq[:,:,0:1]
    
    # CRITICAL: concatenate features, not separate args
    x = torch.cat([torch.cat([Zc, Zq], 1), torch.cat([Tc, Tq], 1), torch.cat([Yc, torch.zeros_like(Yq_true)], 1)], -1)
    
    opt.zero_grad()
    loss = loss_fn(model(x)[:,-1:,:], Yq_true)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    opt.step()
    return loss.item()

def train_causal(model, steps=cfg.DO_STEPS, real_Z=None):
    opt = optim.AdamW(model.parameters(), lr=cfg.LR_TABICL, weight_decay=0.01)
    sched = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=steps)
    loss_fn = nn.MSELoss()
    Zt = torch.from_numpy(real_Z).float().to(cfg.DEVICE) if real_Z is not None else None
    losses = []
    print(f"\n Training {steps} steps...")
    for s in range(steps):
        z, t, y = make_batch(real_Z=Zt if s % 2 == 0 else None)
        losses.append(do_step(model, opt, loss_fn, z, t, y))
        sched.step()
        if (s+1) % 100 == 0: print(f"   Step {s+1}: Loss={np.mean(losses[-100:]):.4f}")
    print(f" Final: {np.mean(losses[-50:]):.4f}")
    return losses

loss_hist = train_causal(model, cfg.DO_STEPS, Z)

In [ ]:
# EVALUATION
@torch.no_grad()
def compute_pehe(model, n=20):
    model.eval()
    errs = []
    for _ in range(n):
        z, t, y = make_batch()
        zn = torch.randn_like(z[:,:1,:])
        Zi = torch.cat([z, zn], 1)
        T1 = torch.cat([t, torch.ones_like(t[:,:1,:])], 1)
        T0 = torch.cat([t, torch.zeros_like(t[:,:1,:])], 1)
        Yi = torch.cat([y, torch.zeros_like(y[:,:1,:])], 1)
        y1 = model(torch.cat([Zi, T1, Yi], -1))[:,-1,0]
        y0 = model(torch.cat([Zi, T0, Yi], -1))[:,-1,0]
        errs.append(((y1 - y0 - cfg.SCM_EFFECT)**2).mean().sqrt().item())
    return np.mean(errs)

@torch.no_grad()
def cate_real(model, Zr, bs=16):
    model.eval()
    Zt = torch.from_numpy(Zr).float().to(cfg.DEVICE)
    cates = []
    for i in range(0, len(Zt), bs):
        zb = Zt[i:i+bs]
        B, F = zb.shape
        zc = torch.randn(B, 5, F, device=cfg.DEVICE)
        logits = cfg.SCM_WEIGHT_T * zc[:,:,0] + torch.randn_like(zc[:,:,0])
        tc = torch.bernoulli(torch.sigmoid(logits)).unsqueeze(-1)
        yc = cfg.SCM_EFFECT * tc - cfg.SCM_WEIGHT_Z * zc[:,:,0:1]
        zq = zb.unsqueeze(1)
        def pred(tv):
            Zi = torch.cat([zc, zq], 1)
            Ti = torch.cat([tc, torch.full((B,1,1), tv, device=cfg.DEVICE)], 1)
            Yi = torch.cat([yc, torch.zeros(B,1,1, device=cfg.DEVICE)], 1)
            return model(torch.cat([Zi, Ti, Yi], -1))[:,-1,0]
        cates.append((pred(1.0) - pred(0.0)).cpu())
    return torch.cat(cates).numpy()

pehe = compute_pehe(model, 50)
cate = cate_real(model, Z)
print(f"\n RESULTS:")
print(f"   PEHE: {pehe:.4f}")
print(f"   CATE: mean={cate.mean():.3f}, std={cate.std():.3f}")
print(f"   True τ: {cfg.SCM_EFFECT}")

In [ ]:
# PLOTS
fig, ax = plt.subplots(1, 3, figsize=(15, 4))
ax[0].plot(loss_hist, alpha=0.3)
ma = np.convolve(loss_hist, np.ones(20)/20, mode='valid')
ax[0].plot(range(19, len(loss_hist)), ma, 'r-', lw=2)
ax[0].set_xlabel('Step'); ax[0].set_ylabel('Loss'); ax[0].set_title('Do-Loss')

ax[1].hist(cate, bins=25, alpha=0.7, edgecolor='k')
ax[1].axvline(cate.mean(), color='r', ls='--', lw=2, label=f'Mean: {cate.mean():.2f}')
ax[1].axvline(cfg.SCM_EFFECT, color='g', ls=':', lw=2, label=f'True: {cfg.SCM_EFFECT}')
ax[1].legend(); ax[1].set_xlabel('CATE'); ax[1].set_title('CATE Distribution')

ax[2].scatter(Y_intersynth, cate, alpha=0.5, edgecolors='k')
ax[2].set_xlabel('InterSynth'); ax[2].set_ylabel('CATE'); ax[2].set_title('CATE vs Anatomy')
plt.tight_layout(); plt.show()

In [ ]:
# SAVE
os.makedirs("Data/checkpoints", exist_ok=True)
np.save("Data/latents.npy", Z)
np.save("Data/intersynth.npy", Y_intersynth)
np.save("Data/cate.npy", cate)
torch.save({'state_dict': vae.state_dict(), 'history': vae_history}, "Data/checkpoints/vae.pt")
torch.save({'state_dict': model.state_dict(), 'losses': loss_hist}, "Data/checkpoints/tabicl.pt")
with open("Data/metrics.json", 'w') as f:
    json.dump({'pehe': pehe, 'cate_mean': float(cate.mean()), 'cate_std': float(cate.std()), 'true_tau': cfg.SCM_EFFECT}, f, indent=2)
print("\n Saved!")